In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install dask[complete]

     |████████████████████████████████| 112kB 4.9MB/s 
     |████████████████████████████████| 706kB 6.2MB/s 
ERROR: distributed 2021.4.1 has requirement cloudpickle>=1.5.0, but you'll have cloudpickle 1.3.0 which is incompatible.
ERROR: distributed 2021.4.1 has requirement dask>=2021.03.0, but you'll have dask 2.12.0 which is incompatible.
  Found existing installation: distributed 1.25.3
    Uninstalling distributed-1.25.3:
      Successfully uninstalled distributed-1.25.3


In [ ]:
pip install Flask-WTF

     |████████████████████████████████| 174kB 4.5MB/s 


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask,render_template,session,url_for,redirect,request
import numpy as np 
import os
import werkzeug
from nltk.stem import PorterStemmer
from joblib import load
import pickle
import os ,os.path
from pandas import DataFrame
import pandas as pd
import dask.dataframe as dd
import csv
from flask_wtf import FlaskForm
from wtforms import StringField,SubmitField,RadioField
from flask_wtf.file import FileField, FileRequired, FileAllowed

# ================================================================================================
import spacy

import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import en_core_web_sm

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = en_core_web_sm.load()
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

from sklearn.base import TransformerMixin

def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))

# ====================================================================================
import tensorflow as tf
from tensorflow.keras.models import load_model
model = load_model("lstm_model.h5")

import tempfile

model_dir = tempfile.gettempdir() # gets the path of the temporary folder
version = 1 # version of the specific model
export_path = os.path.join(model_dir, str(version))
if os.path.isdir(export_path):
  print("Found an existing version of the model. Deleting the previous version\n")
  !rm -r {export_path} # terminal command to remove a directory

# Saving the model
tf.keras.models.save_model(
  model, # the trained model
  export_path, # export path for the model
  overwrite = True, # overwrite if existing version found
  include_optimizer = True, # include the model optimizer
  save_format = None,
  options = None
)

# If tf.keras.models.save_model() doesn't work, use the model.save function
model.save(export_path, save_format="tf")
             
print("Model Saved\n")

os.environ['MODEL_DIR'] = model_dir



INFO:tensorflow:Assets written to: /tmp/1/assets
INFO:tensorflow:Assets written to: /tmp/1/assets
Model Saved



In [ ]:
import sys
# We need sudo prefix if not on a Google Colab.
if 'google.colab' not in sys.modules:
  SUDO_IF_NEEDED = 'sudo'
else:
  SUDO_IF_NEEDED = ''

In [ ]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | {SUDO_IF_NEEDED} tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | {SUDO_IF_NEEDED} apt-key add -
!{SUDO_IF_NEEDED} apt update

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0   136k      0 --:--:-- --:--:-- --:--:--  136k
OK
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server amd64 Packages [340 B]
Get:4 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server-universal amd64 Packages [348 B]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [53.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/m

In [ ]:
!{SUDO_IF_NEEDED} apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  tensorflow-model-server
0 upgraded, 1 newly installed, 0 to remove and 85 not upgraded.
Need to get 326 MB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server amd64 tensorflow-model-server all 2.5.0 [326 MB]
Fetched 326 MB in 6s (54.7 MB/s)
Selecting previously unselected package tensorflow-model-server.
(Reading database ... 160690 files and directories currently installed.)
Preparing to unpack .../tensorflow-model-server_2.5.0_all.deb ...
Unpacking tensorflow-model-server (2.5.0) ...
Setting up tensorflow-model-server (2.5.0) ...


In [ ]:
%%bash --bg 
nohup tensorflow_model_server \
  --port=8500\
  --rest_api_port=8501 \
  --model_name=lstm_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1


Starting job # 0 in a separate thread.


In [ ]:
!tail server.log

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import json
import requests
from tensorflow.keras.models import load_model

tokenizer = Tokenizer()

def decode_sentiment(score):
    return "Business Exception" if score>0.5 else "System Exception"

def lstmPredict(exceptions):
  tokenizer.fit_on_texts(exceptions)
  processed = pad_sequences(tokenizer.texts_to_sequences(exceptions),maxlen =30)
  data = json.dumps({"signature_name": "serving_default", "instances": processed.tolist()})
  headers = {"content-type": "application/json"}
  json_response = requests.post('http://localhost:8051/v1/models/lstm_model:predict', data=data, headers=headers)
  print(json_response)
  preds = list(json.loads(json_response.text)['predictions'])
  processed = [decode_sentiment(score) for score in preds]
  lines = list()
  for i in range(len(processed)):
    line = list()
    line.append(processed[i])
    line.append(exceptions[i])
    s=" : "
    s= s.join(line)
    exceptions[i] = s


	

model_file_name = 'model.pkl' 
def piplinePredict(model,exceptions):
	ops = model.predict(exceptions)
	lines = list()
	for i in range(len(ops)):
		line = list()
		line.append(ops[i])
		line.append(exceptions[i])
		s=" : "
		s= s.join(line)
		exceptions[i] = s


def stemming(wds):
	porter = PorterStemmer()
	for i in range(len(wds)):
		wds[i] = porter.stem(wds[i])

key_bus = ['contain','error','fields','mandatory','missing','found','template','file','field', 'input','processed','mail','invite','manually','id']
key_sys = ['internal','exception','memory','expression','code','stage','site','perform','evaluate','cleanup','time','hresult','failed','operation','unable','issue','data','fetch','unexpected','automatically','set','launch']

porter = PorterStemmer()
def brute_force_audit(txts):
	stemming(key_bus)
	stemming(key_sys)
	for i in range(len(txts)):
		sent = txts[i].lower()
		txts[i] = txts[i].split()
		words = sent.split()
		stemming(words)
		Bis_check =  any(item in key_bus for item in words)
		Sys_check =  any(item in key_sys for item in words)
		if Bis_check == True:
			op = ['Business Exception : ']
			for j in txts[i]:
				op.append(j)
			s=" "
			s = s.join(op)
			txts[i] = s
		elif Sys_check == True:
			op = ['System Exception : ']
			for j in txts[i]:
				op.append(j)
			s=" "
			s = s.join(op)
			txts[i] = s


app = Flask(__name__)
app.config['SECRET_KEY'] = 'nlp_classifier'
run_with_ngrok(app) 


@app.route("/",methods = ['GET','POST'] )
def index():
	
	dat = list()
	op=list()
	if request.method =='POST':
		f = request.files['csvfile']
		cf = isinstance(f,werkzeug.datastructures.FileStorage)
		if f.filename!='':
			f.save(os.path.join(os.getcwd(),f.filename))
			fname = str(f.filename)
			if fname !="":
				df = dd.read_csv(fname)
				df = df.iloc[:,0].to_frame()
				df.to_csv('real_input.csv')

				DIR = str(os.getcwd())+"/real_input.csv"
				num =  (len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))
				for i in range(num):
  		 			df1 = pd.read_csv(DIR+'/'+str(i)+'.part')
  		 			dat = dat + list(df1.iloc[:,1])

				ana_type = request.form.getlist('Class')
				if '3' in ana_type:
					brute_force_audit(dat)
					opdf = DataFrame(dat,columns=['Brute Force Results'])
					opdf.to_csv('brute_force_audit_results.csv')
				elif '2' in ana_type:
					model = load(model_file_name)
					piplinePredict(model,dat)
					op_pip_df = DataFrame(dat,columns=['Spacy Classifier Results'])
					op_pip_df.to_csv('spacy_audit_results.csv')
				elif '1' in ana_type:
					lstmPredict(dat)
					op_lstm_df = DataFrame(dat,columns=['LSTM Classifier Results'])
					op_lstm_df.to_csv('lstm_audit_results.csv')

					
					
			else:
				dat.clear()
		else:
			dat.clear()
		
		# check for type of analysis
		# pass the info in dat to that prediction function


	
	return render_template('class_home.html',data = dat)	


if __name__== '__main__':
	
	app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://a68ec9b0ebe8.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [05/May/2021 11:24:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 11:24:58] "GET /favicon.ico HTTP/1.1" 404 -
[2021-05-05 11:25:08,257] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connection.py", line 159, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/usr/local/lib/python3.7/dist-packages/urllib3/util/connection.py", line 80, in create_connection
    raise err
  File "/usr/local/lib/python3.7/dist-packages/urllib3/util/connection.py", line 70, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 354, in _make_r